In [ ]:
import tensorflow as tf
import cv2 
import numpy as np
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [ ]:
##Load Image Create Dataset

In [ ]:
%%bash
#sh ./*.sh

#INSERT HELPER SCRIPTS HERE

In [ ]:
imgDirNorm = 'E:/Normal Train Data/'
imgDirAnom = 'E:/Anomaly Train Data/'

In [ ]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename),0)
        if img is not None:
            images.append(img)
    return images

In [ ]:
def imrotate(img , angle , scale=1):
    (h, w) = img.shape
    center = (w / 2, h / 2)
    M = cv2.getRotationMatrix2D(center, angle, scale)
    return cv2.warpAffine(img, M, (h, w))

In [ ]:
#DATA AUGMENTATION  (OPTIONAL)
def augment_folder(folder):
    augmented_images = []
    images = load_images_from_folder(folder)

    for i in images:
        augmented_images.append(cv2.flip(i, 1))
        augmented_images.append(cv2.flip(i, 0))
        augmented_images.append(cv2.flip(i, -1))
        rotate90 = imrotate(i,90)
        rotate180 = imrotate(i,180)
        rotate270 = imrotate(i,270)
        augmented_images.append(cv2.flip(rotate90, 1))
        augmented_images.append(cv2.flip(rotate180, 0))
        augmented_images.append(cv2.flip(rotate270, -1))
        augmented_images.append(rotate90)
        augmented_images.append(rotate180)
        augmented_images.append(rotate270)
    
    return augmented_images    

In [ ]:
Anomaly = load_images_from_folder(imgDirAnom)
Normal = load_images_from_folder(imgDirNorm)

train_images=[]
train_labels=[]
test_images=[] 
test_labels=[]

ratio = len(Normal)//len(Anomaly)

i=0
j=0

while(i<len(Normal) and j<len(Anomaly)):
    
    if i%ratio:
        if (Normal[i].shape[0]==1024):
            Normal[i] = cv2.resize(Normal[i]  , (512 , 512))  
        train_images.append(np.reshape(Normal[i],(512,512,1)))
        train_labels.append([0])

    if not i%ratio:
        if (Anomaly[j].shape[0]==1024):
            Anomaly[j] = cv2.resize(Anomaly[j]  , (512 , 512))            
        train_images.append(np.reshape(Anomaly[j],(512 , 512 , 1)))
        train_labels.append([1])
        j+=1
        if (Normal[i].shape[0]==1024):
            Normal[i] = cv2.resize(Normal[i]  , (512 , 512)) 
        train_images.append(np.reshape(Normal[i],(512 , 512,1)))
        train_labels.append([0])
        

    i+=1

print(len(Normal))
print(len(Anomaly))

class_names = ['Clean', 'Anomaly Present']

In [ ]:
augmented_images = augment_folder(imgDirAnom)
Anomaly = Anomaly + augmented_images

print(len(Anomaly))

In [ ]:
train_images=np.array(train_images)/255.0
train_labels=np.array(train_labels)/255.0
test_images=np.array(test_images)/255.0 
test_labels=np.array(test_labels)/255.0

In [ ]:
#(train_images2, train_labels2), (test_images2, test_labels2) = datasets.cifar10.load_data()
#print ((train_images2[0]).shape)
#print (type(train_images2))
# Normalize pixel values to be between 0 and 1
#print(train_images2.shape)
#print(train_labels2.shape)

#train_images2, test_images2 = train_images2 / 255.0, test_images2 / 255.0


In [ ]:
test_images=train_images[0:100]
test_labels=train_labels[0:100]

print((train_images[0]).shape)
print(type(train_images))
print(np.array(train_images).shape)
print(np.array(train_labels).shape)
print(len(test_images))
print(len(test_labels))

In [ ]:
'''class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()'''

In [ ]:
inputshape = (512, 512, 1)
model = models.Sequential()
model.add(layers.Conv2D(64, (2, 2), activation='relu', input_shape=inputshape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (1, 1), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(16, (1, 1), activation='relu'))
model.add(layers.MaxPooling2D((5, 5)))
model.add(layers.Conv2D(16, (1, 1), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(2))
model.summary()

In [ ]:
img_file = './model_arch.png'
tf.keras.utils.plot_model(model, to_file=img_file, show_shapes=True, show_layer_names=True)

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=50, 
                    validation_data=(test_images, test_labels))

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)